<a href="https://colab.research.google.com/github/huannawang/Groq-SerpAPI-Line-Bot/blob/main/%E6%9D%B1%E8%8F%AF_611321222_%E8%B3%87%E5%B7%A5%E7%A2%A9%E4%B8%80_%E7%8E%8B%E5%AE%87%E5%AF%B0_%E6%9C%9F%E6%9C%AB%E5%B0%88%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ngrok

In [ ]:
#將這個 Colab 筆記本串接 Google 雲端硬碟
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!mkdir -p /drive
#umount /drive
!mount --bind /content/drive/My\ Drive /drive
!mkdir -p /drive/ngrok-ssh
!mkdir -p ~/.ssh

Mounted at /content/drive


In [ ]:
#將 ngrok 安裝到 Google 雲端硬碟中
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

/drive/ngrok-ssh
--2025-05-16 05:27:44--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 99.83.220.108, 13.248.244.96, 35.71.179.82, ...
Connecting to bin.equinox.io (bin.equinox.io)|99.83.220.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  33.5MB/s    in 0.4s    

2025-05-16 05:27:45 (33.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip


In [ ]:
#使用 ngrok 指令輸入 token
from google.colab import userdata
ngrok_token = userdata.get("ngrokToken")
!/ngrok authtoken {ngrok_token}

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!pip install pyngrok

In [ ]:
#輸入 ngrok token，與 ngrok 進行認證
import getpass

from pyngrok import ngrok, conf

print("Enter your ngrok authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = ngrok_token

# Open a TCP ngrok tunnel to the SSH server
connection_string = ngrok.connect("22", "tcp").public_url

ssh_url, port = connection_string.strip("tcp://").split(":")
print(f" * ngrok tunnel available, access with `ssh root@{ssh_url} -p{port}`")

In [ ]:
#測試用，不用執行
from pyngrok import ngrok

public_url = ngrok.connect(5000)
print(" * ngrok public URL:", public_url)

In [ ]:
#清理port
from pyngrok import ngrok
ngrok.kill()

#Groq提取查詢關鍵字

In [ ]:
!pip install serpapi
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 10.5 MB/s eta 0:00:00


In [ ]:
#設定Groq模型
import os
from groq import Groq
from google.colab import userdata
from openai import OpenAI

Groq_api_key = userdata.get('Groq')
os.environ["OPENAI_API_KEY"] = Groq_api_key
model = "llama3-70b-8192"
base_url="https://api.groq.com/openai/v1"
client = OpenAI(base_url=base_url)

In [ ]:
#模型將使用者的查詢提取出關鍵字
import json

def Analyze_keywords(user_query):
  system_inst = (
      "Parse the user query into a JSON list of keyword strings, "
      "which is suitable for the Google search engine. "
      "For example, query: how to learn python? "
      'Output: {"keywords": ["python", "learn"]}'
  )
  messages = [
      {"role": "system", "content": system_inst},
      {"role": "user", "content": user_query},
  ]

  completion = client.chat.completions.create(
      model=model,
      messages=messages,
      response_format={"type": "json_object"},
  )
  result = json.loads(completion.choices[0].message.content)

  q = " ".join(result["keywords"])

  return q

#SerpAPI


In [ ]:
#設定SerpApi
import serpapi

serpapi_api_key = userdata.get('SerpAPI')
serp_client = serpapi.Client(api_key=serpapi_api_key)

In [ ]:
#將Groq提取完的查詢關鍵字給SerpApi查詢，return多筆查詢結果
def SerpAPI_search(query_keywords):
  result = serp_client.search(q=query_keywords, engine="google", location="Taiwan", hl="zh-tw", gl="tw")
  snippets = [item["snippet"] for item in result["organic_results"]]
  return snippets

#Groq分析查詢結果

In [ ]:
#Groq分析查詢結果的system_prompt
def Groq_Analyze(snippets, user_query):
  system_prompt = """你是一位專業的資訊統整與分析專家，擅長統整與分析多筆資訊。
  請根據提供的資料來統整與分析，讓使用者快速掌握資訊，用簡單、清楚的台灣中文說明。
  重要：請一律使用台灣常用的【繁體中文】回答，禁止使用英文或簡體字。"""
  #Groq分析查詢結果的final_prompt
  final_prompt = "\n\n".join(snippets)
  final_prompt += f"\n\n請根據以上資訊，詳細回答使用者的問題：{user_query}"

  messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": final_prompt},
  ]
  #Groq分析查詢結果
  chat_completion = client.chat.completions.create(
      messages=messages,
      model=model,
  )
  Analysis_result = chat_completion.choices[0].message.content

  return Analysis_result

#Groq對話模式，不執行搜尋

In [ ]:
# 處理一般對話（純聊天、無搜尋）
def Groq_Chat(user_id, user_query):
    system_prompt = """你是一位親切、專業的ai大型語言模型，請用台灣常見的繁體中文回答問題，請勿使用簡體字與英文。"""

    # 取出使用者的對話歷史
    history = conversation_histories.get(user_id, [])

    # 插入 system prompt 在最前面
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": user_query}]

    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
    )
    assistant_reply = chat_completion.choices[0].message.content
    return assistant_reply

# Line bot

In [ ]:
!pip install flask line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 12.7 MB/s eta 0:00:00


In [ ]:
#建立伺服器，會得到一串 ngrok 對應的網址
#前往Line官方帳號Messaging API將此網址貼到"Webhook網址"
#前往Line devrloper的devrlope console 的 Messaging API settings認證
from flask import Flask, request
from pyngrok import ngrok


# 載入 LINE Message API 相關函式庫
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage

import json

app = Flask(__name__)
port = "5000"

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\" ")

# 管理使用者搜尋狀態
search_mode_users = set()
# 記錄每個 user_id 的對話歷史
conversation_histories = {}

@app.route("/", methods=['POST'])
def linebot():
    body = request.get_data(as_text=True)            # 取得收到的訊息內容
    try:
        json_data = json.loads(body)            # json 格式化訊息內容
        access_token = userdata.get('ChannelAccessToken')
        secret = userdata.get('ChannelSecret')
        line_bot_api = LineBotApi(access_token)       # 確認 token 是否正確
        handler = WebhookHandler(secret)           # 確認 secret 是否正確
        signature = request.headers['X-Line-Signature']   # 加入回傳的 headers
        handler.handle(body, signature)            # 綁定訊息回傳的相關資訊

        event = json_data['events'][0]
        msg = event['message']['text']           # 取得 LINE 收到的文字訊息
        tk = event['replyToken']              # 取得回傳訊息的 Token
        user_id = event['source']['userId']

        # 初始化該使用者的對話歷史（如果還沒有的話）
        if user_id not in conversation_histories:
            conversation_histories[user_id] = []

        #使用者按下搜尋按鈕（傳出特定訊息）
        if msg == "我要開始輸入問題囉":
            search_mode_users.add(user_id)
            line_bot_api.reply_message(tk, TextSendMessage("請輸入你想查詢的問題，我會幫你搜尋並分析"))

        #如果使用者在搜尋模式中
        if user_id in search_mode_users:
            query_keywords = Analyze_keywords(msg)
            SerpAPI_search_result = SerpAPI_search(query_keywords)
            answer = Groq_Analyze(SerpAPI_search_result, msg)
            search_mode_users.remove(user_id)
        else:#一般對話模式（不搜尋）
            answer = Groq_Chat(user_id, msg)

        # 更新對話歷史
        conversation_histories[user_id].append({"role": "user", "content": msg})
        conversation_histories[user_id].append({"role": "assistant", "content": answer})

        # 保留最近 50 輪（100筆 message）
        if len(conversation_histories[user_id]) > 100:
            conversation_histories[user_id] = conversation_histories[user_id][-100:]

        line_bot_api.reply_message(tk, TextSendMessage(answer))  # 回傳訊息


    except:
        print(body)                      # 如果發生錯誤，印出收到的內容
    return 'OK'                          # 驗證 Webhook 使用，不能省略
if __name__ == "__main__":
  app.run()